In [21]:
from urllib.request import urlopen as uReq #urllib will get the page itself
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import nltk
import heapq
import requests
import sys
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import wikipedia
from selenium import webdriver
import os
import urllib.request
import urllib.parse
from tqdm import tqdm
import itertools

In [22]:
therapeutic_justice_type = []

In [23]:
website = 'https://www.judiciary.gov.sg/family'
user_agent = {'User-agent' : 'Mozilla/5.0'}
page = requests.get(website, headers=user_agent)
soup = BeautifulSoup(page.text, 'lxml')
fact_cards = soup.find_all('a', {'class': 'nav-link'})
tj_dict = {}
all_types = []

for fact_card in fact_cards[8:]:
    tj_type = fact_card.get_text().split('\n')[1]
    tj_desc = fact_card.get_text().split('\n')[2]
    tj_link = fact_card['href']
    tj_dict[tj_type] = [tj_desc, tj_link]
    all_types.append(tj_type)

tj_dict

{'Adoption': ['Adoption is the process where parties apply to become the adoptive parents of a child. The Family Courts hear applications for adoption orders.',
  '/family/adoption'],
 'Care and protection for children and young persons': ['The Youth Courts may make care and protection orders to ensure that children in need of care and protection are safe.',
  '/family/care-protection-children-young-persons'],
 'Divorce': ['An application for divorce is a legal procedure to end a marriage. The Family Courts hear divorce applications.',
  '/family/divorce'],
 'Deputyship': ['The Family Courts can appoint a deputy to make decisions for a person who lacks mental capacity. Find out what it means and the ways to apply.',
  '/family/deputyship'],
 'Direct judicial communication for international family proceedings affecting children': ['Information for judges in other jurisdictions on whom to contact regarding direct judicial communication with the Family Justice Courts for international fam

In [24]:
desc_dict = {}
tdesc_dict = {}
ndesc_dict = {}
ddesc_dict = {}
link_dict = {}
row_dict = {}
nav_link_list = []

for type in all_types:
    row_link_list = []
    with open(type + ".txt", "w") as f:
        print(type)
        next_page = 'https://www.judiciary.gov.sg' + tj_dict[type][1]
        page = requests.get(next_page, headers=user_agent)
        soup = BeautifulSoup(page.text, 'lxml')
        content_desc = soup.find_all('div', {'class': 'sf-content-block'})
        accr_desc = soup.find_all('div', {'class': 'accordion-wrapper'})
        # This gets everything in a page, including the p and ul tags
        for desc in content_desc[2:-6]:
            desc_text = desc.get_text()
            f.write(desc_text.replace('\n', ' '))
            # print(desc_text.replace('\n', ' '))
            for link in desc.find_all('a'):
                desc_dict[link.string] = link['href']

        # This gets all the links in the accordion
        for desc in accr_desc:
            for link in desc.find_all('a'):
                link_dict[link.string] = link['href']
            
        if type =="Adoption" or type == "Protection for vulnerable adults" :
            timeline_desc = soup.find_all('div', {'class': 'timeline'})
            for t_desc in timeline_desc:
                timeline_text = t_desc.get_text()
                f.write(timeline_text.replace('\n', ' '))
                # print(timeline_text.replace('\n', ' '))
                for link in t_desc.find_all('a'):
                    tdesc_dict[link.string] = link['href']

        elif type == "Care and protection for children and young persons" or type == "Guardianship":
            nav_desc = soup.find_all('li', {'class': 'nav-item w-100'})
            for n_desc in nav_desc:
                # print(n_desc)
                nav_text = n_desc.get_text()
                f.write(nav_text.replace('\n', ' '))
                # print(nav_text.replace('\n', ' '))
                for link in n_desc.find_all('a'):
                    nav_link_list.append(link['href'])

        else:
            div_desc = soup.find_all('div', {'class': 'row'})
            if type != "Direct judicial communication for international family proceedings affecting children" and type != "Syariah Court orders":
                type_list = []
                for d_desc in div_desc[:-2]:
                    div_text = d_desc.get_text()
                    f.write(div_text.replace('\n', ' '))
                    # print(div_text.replace('\n', ' '))
                    type_list.append(list(filter(None,div_text.split("\n"))))
                    for link in d_desc.find_all('a'):
                        row_link_list.append(link['href'])
                    row_dict[type] = row_link_list

Adoption
Care and protection for children and young persons
Divorce
Deputyship
Direct judicial communication for international family proceedings affecting children
Family Justice Courts appeals
Family guidance for children and young persons
Guardianship
International child abduction
Probate and administration
Protection against family violence
Protection for vulnerable adults
Maintenance
Mediation and counselling in the Family Justice Courts
Syariah Court orders


In [25]:
def get_txt(txt_name, link):
    with open(txt_name, "a") as f:
        next_page = 'https://www.judiciary.gov.sg' + link
        page = requests.get(next_page, headers=user_agent)
        soup = BeautifulSoup(page.text, 'lxml')
        content_desc = soup.find_all('div', {'class': 'sf-content-block'})
        for desc in content_desc[2:-6]:
            desc_text = desc.get_text()
            f.write(desc_text)
            # for link in desc.find_all('a'):
            #     desc_dict[link.string] = link['href']

For Adoption

In [26]:
adoption_int_links = dict(itertools.islice(tdesc_dict.items(), 4))
for link in adoption_int_links.values():
    get_txt('adoption_internal.txt', link)

For Protection for vulnerable adults

In [27]:
protection_vulnerable_int_links =  dict(itertools.islice(tdesc_dict.items(), len(tdesc_dict)-2, None))
for link in protection_vulnerable_int_links.values():
    get_txt('protection_vulnerable_internal.txt', link)

For Care and protection for children and young persons

In [28]:
for link in nav_link_list[:2]:
    get_txt('care_protection_children_internal.txt', link)

Guardianship

In [29]:
for link in nav_link_list[2:]:
    get_txt('guardianship_internal.txt', link)

Divorce, Deputyship, Family justice courts appeals, Family guidance for children and young persons, International child abduction, Probate and administration, Protection against family violence, Maintenance, Mediation and counselling in the family justice courts

In [30]:
# iterate through the dictionary and get the key and value
for key, list_of_links in row_dict.items():
    key = key.replace(" ", "_").lower()
    for link in list_of_links:
        get_txt(key + '_internal.txt', link)

KeyboardInterrupt: 

Civil Law

In [ ]:
website = 'https://www.judiciary.gov.sg/civil'
user_agent = {'User-agent' : 'Mozilla/5.0'}
page = requests.get(website, headers=user_agent)
soup = BeautifulSoup(page.text, 'lxml')
fact_cards = soup.find_all('a', {'class': 'nav-link'})
tj_dict = {}
all_types = []

for fact_card in fact_cards[8:]:
    tj_type = fact_card.get_text().split('\n')[1]
    tj_desc = fact_card.get_text().split('\n')[2]
    tj_link = fact_card['href']
    tj_dict[tj_type] = [tj_desc, tj_link]
    all_types.append(tj_type)

tj_dict

{'Admiralty proceedings (from 1 April 2022)': ['Admiralty disputes involve ships or maritime activities. The High Court (General Division) can hear certain claims against ships, shipowners or ship charterers.',
  '/civil/admiralty-proceedings-(from-1-april-2022)'],
 'Appeals under the Protection from Online Falsehoods and Manipulation Act (POFMA) (from 1 April 2022)': ['Find out how to appeal to the High Court (General Division) against a direction or declaration under the Protection from Online Falsehoods and Manipulation Act.',
  '/civil/appeals-under-the-protection-from-online-falsehoods-and-manipulation-act-(pofma)-(from-1-april-2022)'],
 'Bankruptcy': ['Any individual or firm that is unable to repay any debt of at least $15,000 may be declared a bankrupt by the High Court (General Division). ',
  '/civil/bankruptcy'],
 'Bills of sale: registration and renewal (from 1 April 2022)': ['A bill of sale is a legal document that records the transfer of ownership of an asset. Register or 

In [ ]:
desc_dict = {}
tdesc_dict = {}
ndesc_dict = {}
ddesc_dict = {}
link_dict = {}
row_dict = {}
nav_link_list = []

for type in all_types:
    row_link_list = []
    with open(os.path.join('civil_db', type + ".txt"), "w") as f:
        print(type)
        next_page = 'https://www.judiciary.gov.sg' + tj_dict[type][1]
        page = requests.get(next_page, headers=user_agent)
        soup = BeautifulSoup(page.text, 'lxml')
        content_desc = soup.find_all('div', {'class': 'sf-content-block'})
        accr_desc = soup.find_all('div', {'class': 'accordion-wrapper'})
        # This gets everything in a page, including the p and ul tags
        for desc in content_desc[2:6]:
            desc_text = desc.get_text()
            f.write(desc_text.replace('\n', ' '))
            print(desc_text.replace('\n', ' '))
            for link in desc.find_all('a'):
                desc_dict[link.string] = link['href']

            div_desc = soup.find_all('div', {'class': 'row'})
            if type == "Civil claims (from 1 April 2022)" or type == "Bankruptcy" or type == "Community and neighbour disputes" or type == "Employment claims" or type == "Protection from harassment" or type == "Small claims":
                type_list = []
                for d_desc in div_desc[:-2]:
                    div_text = d_desc.get_text()
                    f.write(div_text.replace('\n', ' '))
                    # print(div_text.replace('\n', ' '))
                    type_list.append(list(filter(None,div_text.split("\n"))))
                    for link in d_desc.find_all('a'):
                        row_link_list.append(link['href'])
                    row_dict[type] = row_link_list        
            

        # This gets all the links in the accordion
        for desc in accr_desc:
            for link in desc.find_all('a'):
                link_dict[link.string] = link['href']
        print('-------------------------------------------------------')
    
            

In [ ]:
def get_txt(type, link):
    with open(os.path.join('civil_db', type + ".txt"), "a") as f:
        next_page = 'https://www.judiciary.gov.sg' + link
        page = requests.get(next_page, headers=user_agent)
        soup = BeautifulSoup(page.text, 'lxml')
        content_desc = soup.find_all('div', {'class': 'sf-content-block'})
        for desc in content_desc[2:-6]:
            desc_text = desc.get_text()
            f.write(desc_text)

Civil claims (from 1 April 2022)

In [ ]:
for key, value in row_dict.items():
    for link in value:
        get_txt(key+"_internal", link)

Criminal law

In [31]:
website = 'https://www.judiciary.gov.sg/criminal'
user_agent = {'User-agent' : 'Mozilla/5.0'}
page = requests.get(website, headers=user_agent)
soup = BeautifulSoup(page.text, 'lxml')
fact_cards = soup.find_all('a', {'class': 'nav-link'})
tj_dict = {}
all_types = []

for fact_card in fact_cards[8:]:
    tj_type = fact_card.get_text().split('\n')[1]
    tj_desc = fact_card.get_text().split('\n')[2]
    tj_link = fact_card['href']
    tj_dict[tj_type] = [tj_desc, tj_link]
    all_types.append(tj_type)

tj_dict

{'Bail': ['An accused person can be released from custody if a bailor posts bail. Find out what it means if you are asked to be a bailor and how to post bail.',
  '/criminal/bail'],
 'Charged with a crime': ['What to expect if you are charged with a criminal offence and have to attend court, including a step-by-step guide to the court process.',
  '/criminal/charged-with-crime'],
 'Charged with a traffic or regulatory offence': ['What happens if you are charged with a traffic or regulatory offence, and what to do if you have to attend Night Court or Traffic Court.',
  '/criminal/charged-with-traffic-regulatory-offence'],
 'Criminal appeals': ["\u200bIf you are not satisfied with the court's decision in your criminal case, you can file an appeal against it. Find out what you need to do.",
  '/criminal/appeal'],
 'Criminal motions': ['A criminal motion is an application to the Supreme Court for a specific purpose, such as an extension of time to file an appeal or a revision of the bail a

In [36]:
desc_dict = {}
tdesc_dict = {}
ndesc_dict = {}
ddesc_dict = {}
link_dict = {}
row_dict = {}
nav_link_list = []

for type in all_types:
    row_link_list = []
    with open(os.path.join('criminal_db', type + ".txt"), "w") as f:
        print(type)
        next_page = 'https://www.judiciary.gov.sg' + tj_dict[type][1]
        page = requests.get(next_page, headers=user_agent)
        soup = BeautifulSoup(page.text, 'lxml')
        content_desc = soup.find_all('div', {'class': 'sf-content-block'})
        accr_desc = soup.find_all('div', {'class': 'accordion-wrapper'})
        # This gets everything in a page, including the p and ul tags
        for desc in content_desc[2:6]:
            desc_text = desc.get_text()
            f.write(desc_text.replace('\n', ' '))
            # print(desc_text.replace('\n', ' '))
            for link in desc.find_all('a'):
                desc_dict[link.string] = link['href']

        if type =="Charged with a crime" or type == "Magistrate's Complaints" or type == 'Bail':
            timeline_desc = soup.find_all('div', {'class': 'timeline'})
            for t_desc in timeline_desc:
                timeline_text = t_desc.get_text()
                f.write(timeline_text.replace('\n', ' '))
                # print(timeline_text.replace('\n', ' '))
                for link in t_desc.find_all('a'):
                    tdesc_dict[link.string] = link['href']

        div_desc = soup.find_all('div', {'class': 'row'})
        if type == "Criminal appeals" or type == "Youth arrest":
            type_list = []
            for d_desc in div_desc[:-2]:
                
                div_text = d_desc.get_text()
                print(div_text)
                f.write(div_text.replace('\n', ' '))
                print(div_text.replace('\n', ' '))
                type_list.append(list(filter(None,div_text.split("\n"))))
                for link in d_desc.find_all('a'):
                    row_link_list.append(link['href'])
                row_dict[type] = row_link_list        
            

        # This gets all the links in the accordion
        for desc in accr_desc:
            for link in desc.find_all('a'):
                link_dict[link.string] = link['href']
        print('-------------------------------------------------------')
    
            

Bail
-------------------------------------------------------
Charged with a crime
-------------------------------------------------------
Charged with a traffic or regulatory offence
-------------------------------------------------------
Criminal appeals


Criminal appeals
​If you are not satisfied with the court's decision in your criminal case, you can file an appeal against it. Find out what you need to do.




  Criminal appeals ​If you are not satisfied with the court's decision in your criminal case, you can file an appeal against it. Find out what you need to do.    




Criminal appeals: overview
You can appeal against the court's decision in a criminal case. Find out about the criminal appeal process, including the possible outcomes.






Criminal appeals against decisions of the State Courts
You may file an appeal either online or over the counter, against a decision in a criminal case heard in the State Courts. Find out the steps to do so.






Criminal appeals against de

In [33]:
def get_txt(type, link):
    with open(os.path.join('criminal_db', type + ".txt"), "a") as f:
        next_page = 'https://www.judiciary.gov.sg' + link
        page = requests.get(next_page, headers=user_agent)
        soup = BeautifulSoup(page.text, 'lxml')
        content_desc = soup.find_all('div', {'class': 'sf-content-block'})
        for desc in content_desc[2:-6]:
            desc_text = desc.get_text()
            f.write(desc_text)

In [37]:
row_dict

{'Criminal appeals': ['/criminal/appeal-overview',
  '/criminal/appeal-state-courts-criminal-case',
  '/criminal/appeal-high-court-criminal-case',
  '/criminal/bail-pending-appeal',
  '/criminal/criminal-appeal-hearing',
  '/criminal/withdraw-appeal'],
 'Youth arrest': []}

In [ ]:
for key, value in row_dict.items():
    for link in value:
        get_txt(key+"_internal", link)